## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
col_lst = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 
           'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[col_lst]

# Show the data types for X_df

X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()
X_train['OverTime'] = le.fit_transform(X_train['OverTime'])
X_test['OverTime'] = le.transform(X_test['OverTime'])

y_df[['Attrition', 'Department']].value_counts()

Attrition  Department            
No         Research & Development    828
           Sales                     354
Yes        Research & Development    133
           Sales                      92
No         Human Resources            51
Yes        Human Resources            12
Name: count, dtype: int64

In [7]:
# Create a StandardScaler
# Fit the StandardScaler to the training data
# Scale the training and testing data

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:
# Create a OneHotEncoder for the Department column
# OneHotEncoding for the department and attrition columns
department_encoder = OneHotEncoder(sparse_output=False)
y_train_department = department_encoder.fit_transform(y_train[['Department']])
y_test_department = department_encoder.transform(y_test[['Department']])

attrition_encoder = OneHotEncoder(sparse_output=False)
y_train_attrition = attrition_encoder.fit_transform(y_train[['Attrition']])
y_test_attrition = attrition_encoder.transform(y_test[['Attrition']])

## Create, Compile, and Train the Model

In [9]:
# Find the number of columns in the X training data
input_shape = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape = (input_shape,), name='input_features')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name = "shared1")(input_layer)
shared2 = layers.Dense(128, activation='relu', name = "shared2")(shared1)


In [10]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name = "department_hidden")(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name = "department_output")(department_hidden)


In [11]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name = "attrition_hidden")(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [12]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features (InputLayer)   │ (None, 10)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared1 (Dense)               │ (None, 64)                │             704 │ input_features[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared2 (Dense)               │ (None, 128)               │           8,320 │ shared1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ department_hidden (Dense)     │ (None, 32)                │           4,128 │ shared2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_hidden (Dense)      │ (None, 32)                │           4,128 │ shared2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ department_output (Dense)     │ (None, 3)                 │              99 │ department_hidden[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_output (Dense)      │ (None, 2)                 │              66 │ attrition_hidden[0][0]     │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Train the model
train_md = model.fit(X_train_scaled, 
                     {'department_output': y_train_department, 'attrition_output': y_train_attrition},
                     epochs=100, batch_size=32)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 909us/step - attrition_output_accuracy: 0.6411 - department_output_accuracy: 0.5506 - loss: 1.5620
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - attrition_output_accuracy: 0.8517 - department_output_accuracy: 0.6348 - loss: 1.2169
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - attrition_output_accuracy: 0.8474 - department_output_accuracy: 0.6422 - loss: 1.1666
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - attrition_output_accuracy: 0.8478 - department_output_accuracy: 0.6517 - loss: 1.1355
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - attrition_output_accuracy: 0.8475 - department_output_accuracy: 0.6534 - loss: 1.1435
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - attrition_output_accuracy: 0.8516 - department_output_accuracy: 0.6791 - loss: 1.1016
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - attrition_output_accuracy: 0.8724 - department_output_accuracy: 0.6514 - loss: 1.0693
Epoch 8/100
3

In [16]:
# Evaluate the model with the testing data
evaluation = model.evaluate(X_test_scaled, 
                            {'department_output': y_test_department, 'attrition_output': y_test_attrition})
print(f"Evaluation Results: {evaluation}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - attrition_output_accuracy: 0.8092 - department_output_accuracy: 0.4840 - loss: 4.1502
Evaluation Results: [3.932821750640869, 0.8152173757553101, 0.49728259444236755]


In [18]:
# Print the accuracy for both department and attrition
print(f"Department Prediction Accuracy: {evaluation[1]}")
print(f"Attrition Prediction Accuracy: {evaluation[2]}")

Department Prediction Accuracy: 0.8152173757553101
Attrition Prediction Accuracy: 0.49728259444236755


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 

1. since there was a class imbalance I would rather get either the recall or f1 score as attrition has only 1 class and that is why the score was so low for attrition. An example would be if 85% of the employess are non attrition models will always predict around 85% 2.sigmoid for attrition as it is a binary classification and softmax for department as it is multiclass 3. ways to improve the model would be to add more hidden layers and experiment with other optimizers than just adam 